In [38]:
import tqdm
import torch
import wandb
import pandas as pd
from sacrebleu import corpus_bleu
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [39]:
wandb.init(project="LoRA-Small-BlenderBot-Model-Seq2Seq-FineTuning-Yo-")

In [40]:
dataStuff = pd.read_csv("../../../dataStuff/QAstuff/AngularQnA.csv")
dataStuff

,Question,Response
0,"What are components in Angular, and why are th...",Components are the fundamental building blocks...
1,Explain how routing works in Angular. What is ...,Routing in Angular manages navigation between ...
2,"What are directives in Angular, and how do the...",Directives are markers in your HTML templates ...
3,Describe the concept of services in Angular. ...,Services are reusable classes that encapsulate...
4,"What is TypeScript, and why is it recommended ...",TypeScript is a superset of JavaScript that ad...
...,...,...
248,Explain the concept of Angular Material and it...,Angular Material is a UI component library bui...
249,Describe the concept of Server-Side Rendering ...,Preboot with SSR in Angular Universal: - Prebo...
250,Discuss the concept of Ahead-of-Time (AOT) Com...,AOT Compilation with Tree-Shaking: - AOT compi...
251,Explain the concept of Angular Zones with NgZo...,NgZone in Angular: - NgZone.run() schedules as...


In [41]:
batchSize = 16
learningRate = 2e-5
epochs = 450

In [42]:
torch.cuda.is_available()

True

In [43]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [44]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot_small-90M").to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot_small-90M")

In [45]:
model.config

BlenderbotSmallConfig {
  "_name_or_path": "facebook/blenderbot_small-90M",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BlenderbotSmallForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 8,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 8,
  "eos_token_id": 2,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id

In [46]:
for name, param in model.named_parameters():
    print(name)

model.shared.weight
model.encoder.embed_positions.weight
model.encoder.layers.0.self_attn.k_proj.weight
model.encoder.layers.0.self_attn.k_proj.bias
model.encoder.layers.0.self_attn.v_proj.weight
model.encoder.layers.0.self_attn.v_proj.bias
model.encoder.layers.0.self_attn.q_proj.weight
model.encoder.layers.0.self_attn.q_proj.bias
model.encoder.layers.0.self_attn.out_proj.weight
model.encoder.layers.0.self_attn.out_proj.bias
model.encoder.layers.0.self_attn_layer_norm.weight
model.encoder.layers.0.self_attn_layer_norm.bias
model.encoder.layers.0.fc1.weight
model.encoder.layers.0.fc1.bias
model.encoder.layers.0.fc2.weight
model.encoder.layers.0.fc2.bias
model.encoder.layers.0.final_layer_norm.weight
model.encoder.layers.0.final_layer_norm.bias
model.encoder.layers.1.self_attn.k_proj.weight
model.encoder.layers.1.self_attn.k_proj.bias
model.encoder.layers.1.self_attn.v_proj.weight
model.encoder.layers.1.self_attn.v_proj.bias
model.encoder.layers.1.self_attn.q_proj.weight
model.encoder.la

In [47]:
loraConfig = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [48]:
model = get_peft_model(model, loraConfig)
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 89,606,144 || trainable%: 2.3404


In [49]:
wandb.watch(model)
wandb.config["learning_rate"] = learningRate
wandb.config["batch_size"] = batchSize

In [50]:
trainData, valData = train_test_split(dataStuff, test_size=0.2, random_state=42)
trainData

,Question,Response
38,Explain the concept of pipes in Angular and ho...,Pipes are functions or classes in Angular that...
143,Discuss the concept of Change Detection in Ang...,Change Detection is a mechanism in Angular tha...
84,Explain the concept of Ahead-of-Time (AOT) com...,AOT compilation pre-compiles your Angular appl...
55,Explain the concept of zoneless change detecti...,Zoneless change detection in Angular is an exp...
217,Describe the concept of Dependency Injection (...,DI Scopes define the lifetime of injected depe...
...,...,...
106,Describe the purpose of `NgModule` factories a...,`NgModule` factories are functions responsible...
14,What are some key features and benefits of usi...,The Angular CLI is a powerful tool that stream...
92,Explain the concept of change detection strate...,Change detection strategies define how Angular...
179,Explain the concept of NgRx and its functional...,NgRx is a popular third-party library that imp...


In [51]:
def prepData(data, max_input_length=512, max_output_length=512):
    input_ids, attention_masks, labels = [], [], []
    for prompt, response in zip(data["Question"], data["Response"]):
        encoded_prompt = tokenizer(
            str(prompt),
            padding='max_length',
            truncation=True,
            max_length=max_input_length,
            return_tensors='pt'
        )
        encoded_response = tokenizer(
            str(response),
            padding='max_length',
            truncation=True,
            max_length=max_output_length,
            return_tensors='pt'
        )
        input_ids.append(encoded_prompt["input_ids"].squeeze())
        attention_masks.append(encoded_prompt["attention_mask"].squeeze())
        labels.append(encoded_response["input_ids"].squeeze())

    return input_ids, attention_masks, labels

In [52]:
trainInputs, trainMasks, trainLabels = prepData(trainData)
valInputs, valMasks, valLabels = prepData(valData)

In [53]:
trainLabels

[tensor([11086,    46,  4118,    65,  2880,    13, 18331,    22, 10999,  1419,
           126, 16711,    19,    13,     7,  6675,     8,    17,  5964,  2953,
             5,    52,  1694,    12, 27529,  2541,   132,    11, 17023,  1419,
           307, 18221,  5278,     7,  6675,     8,    17,  6065,  1835,     5,
         27451,   291,     3,    13, 11086,  3068,    26,   922,  8537,    71,
          5600,   607,  1000,  3566,     6, 21111, 10510,     6,  1916,     6,
            65, 11386,  2264,  3087, 13042,     5,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [54]:
print(type(trainInputs))
print(type(trainInputs[0]))


<class 'list'>
<class 'torch.Tensor'>


In [55]:
trainInputs_ = torch.stack([torch.tensor(seq, dtype=torch.long) for seq in trainInputs]).to(device)

C:\Users\Sid\AppData\Local\Temp\ipykernel_12392\4246888458.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainInputs_ = torch.stack([torch.tensor(seq, dtype=torch.long) for seq in trainInputs]).to(device)


In [56]:
trainInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
trainMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainMasks], batch_first=True, padding_value=0).to(device)
trainLabels = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainLabels], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valMasks], batch_first=True, padding_value=0).to(device)
valLabels = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valLabels], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

C:\Users\Sid\AppData\Local\Temp\ipykernel_12392\3383028139.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
C:\Users\Sid\AppData\Local\Temp\ipykernel_12392\3383028139.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainMasks], batch_first=True, padding_value=0).to(device)
C:\Users\Sid\AppData\Local\Temp\ipykernel_12392\3383028139.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTenso

In [57]:
print(trainMasks.max(), trainInputs.min())
print(f"Max train input ID: {trainInputs.max().item()}, Vocab size: {tokenizer.vocab_size}")
print(f"Max val input ID: {valInputs.max().item()}, Vocab size: {tokenizer.vocab_size}")


tensor(1, device='cuda:0') tensor(0, device='cuda:0')
Max train input ID: 31170, Vocab size: 54944
Max val input ID: 31374, Vocab size: 54944


In [58]:
trainDataset = TensorDataset(trainInputs, trainMasks, trainLabels)
trainDataloader = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)

In [59]:
valDataset = TensorDataset(valInputs, valMasks, valLabels)
valDataloader = DataLoader(valDataset, batch_size=batchSize)

In [60]:
for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
    print(f"Batch {batch_idx} - input_ids shape: {input_ids.shape}, attention_mask shape: {attention_mask.shape}, labels shape: {labels.shape}")

Batch 0 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 1 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 2 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 3 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 4 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 5 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 6 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size([16, 512]), labels shape: torch.Size([16, 512])
Batch 7 - input_ids shape: torch.Size([16, 512]), attention_mask shape: torch.Size(

In [61]:
optimizer = AdamW(model.parameters(), lr=learningRate)
criterion = torch.nn.CrossEntropyLoss()

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [62]:
def train(model, optimizer, criterion, trainDataloader):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
        optimizer.zero_grad()
        input_ids = input_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        labels = labels.long().to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == labels).sum().item()
    return total_loss / len(trainDataloader), correct / len(trainDataset)

In [63]:
def validate(model, criterion, valDataloader):
    model.eval()
    total_loss = 0
    correct_sequences = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch_idx, (input_ids, attention_mask, labels) in enumerate(valDataloader):
            input_ids = input_ids.long().to(device)
            attention_mask = attention_mask.long().to(device)
            labels = labels.long().to(device)
            input_ids[input_ids >= tokenizer.vocab_size] = tokenizer.pad_token_id
            labels[labels >= tokenizer.vocab_size] = tokenizer.pad_token_id
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            correct_sequences += (predictions == labels).all(dim=1).sum().item()
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct_sequences / len(valDataloader.dataset)
    flattened_preds = [pred for seq in all_preds for pred in seq]
    flattened_labels = [label for seq in all_labels for label in seq]
    precision = precision_score(flattened_labels, flattened_preds, average="weighted", zero_division=0)
    recall = recall_score(flattened_labels, flattened_preds, average="weighted", zero_division=0)
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in all_preds]
    decoded_labels = [[tokenizer.decode(label, skip_special_tokens=True)] for label in all_labels]

    bleu_score = corpus_bleu(decoded_preds, decoded_labels)

    return total_loss / len(valDataloader), accuracy, precision, recall, bleu_score


In [64]:
for epoch in tqdm.trange(epochs, colour = "red", desc = "Epoch(s)"):
  train_loss, train_acc = train(model, optimizer, criterion, trainDataloader)
  val_loss, val_acc, val_precision, val_recall, bleu_score = validate(model, criterion, valDataloader)
  bleu_score = bleu_score.score
  dataDict = {
    "Epoch": epoch+1,
    "Train Loss": train_loss,
    "Train Accuracy": train_acc,
    "Val Loss": val_loss,
    "Val Accuracy": val_acc,
    "Val Precision": val_precision,
    "Val Recall": val_recall,
    "BLEU Score": bleu_score
    }
  print(dataDict)
  wandb.log(dataDict)
  model.save_pretrained("tunedModels/trainedSmallBlenderBotLoRA - Epoch {}".format(epoch))

Epoch(s):   0%|          | 0/450 [00:00<?, ?it/s]

{'Epoch': 1, 'Train Loss': 10.551042336684008, 'Train Accuracy': 25.861386138613863, 'Val Loss': 9.952340126037598, 'Val Accuracy': 0.0, 'Val Precision': 0.06460610211265769, 'Val Recall': 0.05564491421568627, 'BLEU Score': 1.10324035058962}


Epoch(s):   0%|          | 1/450 [00:39<4:57:35, 39.77s/it]

{'Epoch': 2, 'Train Loss': 10.221856337327223, 'Train Accuracy': 25.96039603960396, 'Val Loss': 9.773019313812256, 'Val Accuracy': 0.0, 'Val Precision': 0.06518288735316359, 'Val Recall': 0.05572150735294118, 'BLEU Score': 1.019278385434927}


Epoch(s):   0%|          | 2/450 [01:19<4:54:40, 39.47s/it]

...

Epoch(s): 100%|█████████▉| 448/450 [4:48:01<01:18, 39.26s/it]

{'Epoch': 449, 'Train Loss': 8.55858854147104, 'Train Accuracy': 43.024752475247524, 'Val Loss': 8.631346464157104, 'Val Accuracy': 0.0, 'Val Precision': 0.07678730841956045, 'Val Recall': 0.08103553921568628, 'BLEU Score': 3.062705249325256}


Epoch(s): 100%|█████████▉| 449/450 [4:48:41<00:39, 39.47s/it]

{'Epoch': 450, 'Train Loss': 8.551233218266415, 'Train Accuracy': 43.37623762376238, 'Val Loss': 8.631473779678345, 'Val Accuracy': 0.0, 'Val Precision': 0.07675132551853517, 'Val Recall': 0.08118872549019608, 'BLEU Score': 3.0469581710975486}


Epoch(s): 100%|██████████| 450/450 [4:49:21<00:00, 38.58s/it]


In [65]:
model.save_pretrained("trainedSmallBlenderBotLoRA")